<a href="https://colab.research.google.com/github/YaninaK/anomaly-detection/blob/b1/notebooks/02_Duplicates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обнаружение аномалий в начислениях за тепловую энергию

## 2. Равные значения показаний в течение нескольких расчетных периодов.

### Примеры аномалий

Виды аномалий по показаниям приборов учёта тепловой энергии, которые необходимо выявлять (кроме объектов с видом энергопотребления ГВС (централ):

1. нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель);

2. равные значения показаний в течение нескольких расчетных периодов;

3. снижение/рост показаний в отдельные месяцы по сравнению с показаниями за предыдущие периоды по данному объекту (с учётом фактической температуры наружного воздуха и количества отопительных дней в месяце);

4. аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами (только для типов объекта «Многоквартирный дом») по критериям:
  - год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
  - этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
  - площадь (±10%),
  - наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).



In [1]:
initiate = False
if initiate:
  !git init -q
  !git clone -b b1  https://github.com/YaninaK/anomaly-detection.git -q

  from google.colab import drive
  drive.mount('/content/drive')

  !unzip -u -q /content/drive/MyDrive/ML_projects/08_anomaly_detection/data/01_raw/task#3.zip -d /content/anomaly-detection/data/01_raw

%cd /content/anomaly-detection

/content/anomaly-detection


In [2]:
import os
import sys

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "src", "anomaly_detection"))

In [3]:
from data.make_dataset import load_data
from features.duplicated import equal_values_identification_pipeline

In [4]:
import warnings
warnings.filterwarnings('ignore')

## 1. Чтение данных

In [5]:
%%time
save = False
data, temperature, buildings = load_data(save=save)

CPU times: user 1.03 s, sys: 54 ms, total: 1.08 s
Wall time: 1.16 s


## 2. Равные значения показаний в течение нескольких расчетных периодов

In [6]:
%%time
save = True
completely_duplicated, equal_values = equal_values_identification_pipeline(
    data, buildings, temperature, save=save
)

CPU times: user 3.6 s, sys: 118 ms, total: 3.71 s
Wall time: 3.89 s


### 2.1 Дублирующиеся входные данные

In [7]:
completely_duplicated

,index,Подразделение,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал",Период потребления
9481,636,Уфа,20-128030,ГВС-ИТП,"г Уфа, ул. Ладыгина, д.11",Многоквартирный дом,2022-08-01,17.848,2022-07-01
9482,637,Уфа,20-128030,ГВС-ИТП,"г Уфа, ул. Ладыгина, д.11",Многоквартирный дом,2022-08-01,17.848,2022-07-01
27501,1190,Уфа,114390,ГВС-ИТП,"г Уфа, ул. Мира, д.33",Многоквартирный дом,2022-05-01,101.094,2022-04-01
27502,1191,Уфа,114390,ГВС-ИТП,"г Уфа, ул. Мира, д.33",Многоквартирный дом,2022-05-01,101.094,2022-04-01
42295,2933,Уфа,2300 N CF944,None,"г Уфа, ул. Трамвайная, д.3",Другое строение,2021-12-01,8213.300,2021-11-01
42353,2991,Уфа,2300 N CF944,None,"г Уфа, ул. Трамвайная, д.3",Другое строение,2021-12-01,8213.300,2021-11-01
73123,2167,Уфа,00273416,ГВС-ИТП,"г Уфа, ул. Комсомольская, д.33",Другое строение,2023-03-01,142.989,2023-02-01
73124,2168,Уфа,00273416,ГВС-ИТП,"г Уфа, ул. Комсомольская, д.33",Другое строение,2023-03-01,142.989,2023-02-01


### 2.2. Равные значения показаний в течение нескольких расчетных периодов

In [8]:
print(f"Число записей с равными значениями показаний: {equal_values.shape[0]}")
print(f"Число адресов с равными значениями показаний: {equal_values['Адрес объекта'].nunique()}\n")
equal_values.tail(10)

Число записей с равными значениями показаний: 242
Число адресов с равными значениями показаний: 94



,index,Подразделение,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал",Период потребления
14104,4145,Уфа,1524327,None,"г Уфа, ул. Энтузиастов, д.14, Подобъект №984513",Многоквартирный дом,2022-03-01,1.000,2022-02-01
48849,2746,Уфа,1524327,None,"г Уфа, ул. Энтузиастов, д.14, Подобъект №984513",Многоквартирный дом,2022-04-01,1.000,2022-03-01
43100,3738,Уфа,"00535101751, 00533101751",None,"г Уфа, ул. Энтузиастов, д.16, Подобъект №984098",Многоквартирный дом,2021-12-01,2.500,2021-11-01
23991,4292,Уфа,"00535101751, 00533101751",None,"г Уфа, ул. Энтузиастов, д.16, Подобъект №984098",Многоквартирный дом,2022-12-01,2.500,2022-11-01
4284,4294,Уфа,65896908,None,"г Уфа, ул. Энтузиастов, д.16, Подобъект №984816",Многоквартирный дом,2023-04-01,2.519,2023-03-01
34930,4285,Уфа,65896908,None,"г Уфа, ул. Энтузиастов, д.16, Подобъект №984816",Многоквартирный дом,2023-05-01,2.519,2023-04-01
70838,4069,Уфа,101433,None,"г Уфа, ул. Юрия Гагарина, д.20",Другое строение,2021-11-01,11.009,2021-10-01
39280,4231,Уфа,101433,None,"г Уфа, ул. Юрия Гагарина, д.20",Другое строение,2022-11-01,11.009,2022-10-01
65954,3524,Уфа,1981,None,"р-н Уфимский, д Лекаревка",Другое строение,2022-01-01,210.000,2021-12-01
48735,2632,Уфа,1981,None,"р-н Уфимский, д Лекаревка",Другое строение,2022-04-01,210.000,2022-03-01
